# Iteration 1

In [1]:
!pip install -q -U transformers bitsandbytes

In [2]:
import json
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
import re

In [9]:
DEBUG_LIMIT = None
MODEL_ID = "Qwen/Qwen2.5-32B-Instruct"
INPUT_FILE = "zno.test.jsonl"
OUTPUT_FILE = "submission.csv"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    trust_remote_code=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

model-00004-of-00017.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00006-of-00017.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

...

model-00015-of-00017.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00016-of-00017.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00017-of-00017.safetensors:   0%|          | 0.00/3.10G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [16]:
data = []
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

In [ ]:
if DEBUG_LIMIT:
    data = data[:DEBUG_LIMIT]
    print(f"Running only first {DEBUG_LIMIT} samples!")

In [ ]:
def format_prompt(item):
    question = item['question']
    options_text = ""
    valid_markers = []

    for ans in item['answers']:
        options_text += f"{ans['marker']}) {ans['text']}\n"
        valid_markers.append(ans['marker'])

    messages = [
        {
            "role": "system",
            "content": "Ви екзаменатор. Ваше завдання визначити правильну відповідь. Виведіть ТІЛЬКИ одну букву (А, Б, В, Г або Д) без пояснень і без крапок."
        },
        {
            "role": "user",
            "content": f"{question}\n\nВаріанти:\n{options_text}\nВідповідь:"
        }
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return text, valid_markers

In [17]:
results = []

for i, item in enumerate(tqdm(data)):
    prompt, valid_markers = format_prompt(item)

    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=5,
            temperature=0.1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    raw_answer = full_response.split("Відповідь:")[-1].strip()

    predicted_letter = "А"

    clean = re.sub(r"[^А-Яа-яA-Za-z]", "", raw_answer)

    if clean and clean in valid_markers:
        predicted_letter = clean
    else:
        match = re.search(r'\b([АБВГД])\b', raw_answer)
        if match:
            predicted_letter = match.group(1)

    results.append({"id": item['id'], "correct_answers": predicted_letter})

    if i < 5 or i % 50 == 0:
        tqdm.write(f"\n[DEBUG ID {item['id']}]")
        tqdm.write(f"Question: {item['question']}")
        tqdm.write(f"Raw Output: '{raw_answer}' -> Parsed: {predicted_letter}")
        tqdm.write("-" * 30)

  0%|          | 1/751 [00:00<03:48,  3.28it/s]


[DEBUG ID 0]
Question: «Сміхом крізь сльози» можна схарактеризувати зображене в обох творах рядка
Raw Output: 'assistant
А' -> Parsed: А
------------------------------


  0%|          | 2/751 [00:00<03:40,  3.40it/s]


[DEBUG ID 1]
Question: Удовин син, мати, сестра, кохана – ключові образи твору
Raw Output: 'assistant
В' -> Parsed: В
------------------------------


  0%|          | 3/751 [00:00<03:56,  3.16it/s]

...


[DEBUG ID 650]
Question: Ознаки постмодернізму притаманні рядкам
Raw Output: 'assistant
В' -> Parsed: В
------------------------------


 93%|█████████▎| 701/751 [03:44<00:15,  3.17it/s]


[DEBUG ID 700]
Question: **Проаналізуйте художній твір і виконайте завдання до нього.**


***О панно Інно! Липень. Спека.***


***Ви тут. Але Ви так далеко.***


***Десь мерехтить дівоча мрія,***


***Як білий сон на темних віях.***


***В цей білий сон, як в танець білий,***


***Прийде хтось несказанно милий***


***І поведе у світ любові,***


***Як заполоч на рушникові,***


***Як ружі цвіт – на білосніжність,***


***Зійде враз невимовна ніжність.***


***Цей липень. Липи. Білопінно.***


***І тихий шепіт:***


***— Інно, Інно…***


Образи в поезії є алюзією на твір з такими рядками
Raw Output: 'assistant
Г' -> Parsed: Г
------------------------------


100%|██████████| 751/751 [03:59<00:00,  3.14it/s]


[DEBUG ID 750]
Question: Прочитайте уривок джерела.


«*У Московщині панує найогидніше рабство. Там немає і бути не може нічого власного, бо все є власністю царя. Московські бояри титулують себе “рабами царськими”. Увесь народ московський є рабом. У Московії продають людей на базарі, як у нас худобу. Приєднатися до такого народу – це гірше, як скочити живим у вогонь…*»


Такими словами аргументував полковник Іван Богун свою позицію щодо недоцільності укладання
Raw Output: 'assistant
А' -> Parsed: А
------------------------------


In [ ]:
df = pd.DataFrame(results)
df.to_csv(OUTPUT_FILE, index=False)
print(f"\nFinished! Saved to {OUTPUT_FILE}")
print(df.head())


✅ Finished! Saved to submission.csv
   id correct_answers
0   0               А
1   1               В
2   2               Г
3   3               Д
4   4               А
